In [3]:
import subprocess
import os
import re
import pyHGSDSSAT.ModelPrep as mp
import pyHGSDSSAT.CreateMapping as cm
import pyHGSDSSAT.ModelInfoExchange as ie
import time
import glob

In [ ]:
files = glob.glob(r"C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1\dssat\1\data")
for f in files:
    os.remove(f)

In [4]:
working_HGS_dir = r"C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1\hgs"
working_DSSAT_dir = r"C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1\dssat"
mapping_dir= r"C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1\mapping"
copuled_dir= r"C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1"
# Get a fixed list of relevant model names at the start
model_name = 'lys_e'
model_files = [
    re.match(r'(lys_e_day)(\d+)\.grok', f).groups()
    for f in os.listdir(working_HGS_dir)
    if re.match(r'lys_e_day\d+\.grok', f)
]
#sort the list of model files
sorted_model_names = [f"{prefix}{int(day)}" for prefix, day in sorted(model_files, key=lambda x: int(x[1]))]
final_simulation_day= len(sorted_model_names)
mapping=ie.ExposeMappingVariables(copuled_dir)

# create bfx file for hgs
output_file = working_HGS_dir +'/batch.pfx'
with open(output_file, 'w') as f:
    f.write(sorted_model_names[0] + '\n')
#run grok and phgs subprocess
subprocess.run(['cmd', '/c', 'grok'], cwd=working_HGS_dir) #, stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)
subprocess.run(['cmd', '/c', 'phgs'], cwd=working_HGS_dir) #stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)
#check what is running and remove from the list
print(f"Processing model: {sorted_model_names[0]}")
# run create DRN function

print(mapping)
ie.GenerateDSSATDailyDRNHGSNodalFlux(mapping,0,model_name,working_HGS_dir,working_DSSAT_dir)
# run all existing DSSAT models for HGS zones 
#for DSSAT_model_name in os.listdir(working_DSSAT_dir):
#    DSSAT_dict=working_DSSAT_dir + "\\" + DSSAT_model_name



DSSAT_dict=working_DSSAT_dir + "\\1"
print(DSSAT_dict)
exe_path = r"C:\Users\glogow0000\HGS-DSSAT\codebase\DSSAT\Coup\DSSAT\build\bin\Debug\dscsm048.exe"
process = subprocess.Popen(
    [exe_path, "C", "SWSW7501.WHX", "1"],
    cwd=DSSAT_dict
    )

print("Process started, PID:", process.pid)
#open loop for dssat subroutine

for i in range(1,final_simulation_day):
    # create bfx file for hgs
    output_file = working_HGS_dir +'/batch.pfx'
    with open(output_file, 'w') as f:
        f.write(model_name + '_day' + str(i))
    print(f"Starting processing for model: {model_name}")
    DSSAT_check = f"{i}.txt"
    full_path = os.path.join(DSSAT_dict, DSSAT_check)
    while not os.path.isfile(full_path):
        print(f"File not found: {full_path}. Waiting 5 seconds...")
        time.sleep(1)
        # If I exceeds final simulation day, break the while loop
        if i > final_simulation_day:
            print(f"Reached final simulation day for {model_name}. Moving to next model.")
            break       
    print(working_HGS_dir)
    time.sleep(1)
    # Call your processing function
    ie.GenerateNodalFluxTimeValueTableDSSATET(mapping, i, working_HGS_dir, working_DSSAT_dir)
    time.sleep(2)
    #run grok and phgs subprocess
    subprocess.run(['cmd', '/c', 'grok'], cwd=working_HGS_dir) #, stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)
    subprocess.run(['cmd', '/c', 'phgs'], cwd=working_HGS_dir) #stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)
    print(f"Processing model: {model_name} {i}")
    time.sleep(1)
    #run create DRN function
    ie.GenerateDSSATDailyDRNHGSNodalFlux(mapping,i,model_name,working_HGS_dir,working_DSSAT_dir)



Processing model: lys_e_day0
{'dlhl': {1: 12, 2: 11, 3: 10, 4: 9, 5: 8, 6: 7, 7: 6, 8: 5, 9: 4}, 'hldl': {12: 1, 11: 2, 10: 3, 9: 4, 8: 5, 7: 6, 6: 7, 5: 8, 4: 9}, 'hnsdb': {13: 1, 12: 2, 11: 3, 10: 4, 9: 5, 8: 6, 7: 7, 6: 8, 5: 9, 4: 10}, 'dlhnsud': {1: [13, 12], 2: [12, 11], 3: [11, 10], 4: [10, 9], 5: [9, 8], 6: [8, 7], 7: [7, 6], 8: [6, 5], 9: [5, 4]}, 'hncvdm': {1: [1, 0.25], 2: [1, 0.5], 3: [1, 0.25], 4: [1, 0.5], 5: [1, 1.0], 6: [1, 0.5], 7: [1, 0.25], 8: [1, 0.5], 9: [1, 0.25]}, 'dm_area_shp': {1: [4.0, 'c:\\Users\\glogow0000\\HGS-DSSAT\\model\\coupled_v1\\mapping\\cv_dssatmod_1.shp']}}
C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1\dssat\1
Process started, PID: 23580
Starting processing for model: lys_e
File not found: C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1\dssat\1\1.txt. Waiting 5 seconds...


C:\Users\glogow0000\HGS-DSSAT\codebase\pyHGSDSSAT\pyHGSDSSAT\ModelInfoExchange.py:171: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(nfmb_path, skiprows = start_line, names = vars, delim_whitespace=True)


C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1\hgs
[4, 5, 6, 7, 8, 9, 10, 11, 12, 13] [1, 2, 3, 4, 5, 6, 7, 8, 9]
C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1\dssat\1\ET.OUT
   @YEAR  DOY  DAS  SRAA  TMAXA  TMINA  REFA   EOAA  EOPA   EOSA  ...   ES2D  \
0   1975  132    1  23.6   11.0    5.0 -99.0  3.496   0.0  3.496  ...  0.563   

    ES3D   ES4D  ES5D   ES6D   ES7D   ES8D   ES9D  TRWUD  TWUPD  
0  0.517  0.321  0.25  0.209  0.211  0.357  0.299    0.0    0.0  

[1 rows x 35 columns]
   day    1    2    3    4    5    6    7    8    9  ...   11   12   13   14  \
0    1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   

    15   16   17   18   19   20  
0  0.0  0.0  0.0  0.0  0.0  0.0  

[1 rows x 21 columns]
{1: {1: 0.769, 2: 0.563, 3: 0.517, 4: 0.321, 5: 0.25, 6: 0.209, 7: 0.211, 8: 0.357, 9: 0.299}}
0 4
0 0.299
0 0.299
0 0.299
0 0.299
0 0.299
0 0.299
0 0.299
0 0.299
0 0.299
1 5
1 0.357 0.299
1 0.357 0.299
1 0.357 0.299
1 0.357 0.299
1 0.357 0.299
1 0.357 0.2

C:\Users\glogow0000\HGS-DSSAT\codebase\pyHGSDSSAT\pyHGSDSSAT\ModelInfoExchange.py:171: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(nfmb_path, skiprows = start_line, names = vars, delim_whitespace=True)


Starting processing for model: lys_e
File not found: C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1\dssat\1\2.txt. Waiting 5 seconds...
C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1\hgs
[4, 5, 6, 7, 8, 9, 10, 11, 12, 13] [1, 2, 3, 4, 5, 6, 7, 8, 9]
C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1\dssat\1\ET.OUT
   @YEAR  DOY  DAS  SRAA  TMAXA  TMINA  REFA   EOAA  EOPA   EOSA  ...   ES2D  \
0   1975  132    1  23.6   11.0    5.0 -99.0  3.496   0.0  3.496  ...  0.563   
1   1975  133    2  21.3   11.0    5.0 -99.0  3.226   0.0  3.226  ...  0.504   

    ES3D   ES4D   ES5D   ES6D   ES7D   ES8D   ES9D  TRWUD  TWUPD  
0  0.517  0.321  0.250  0.209  0.211  0.357  0.299    0.0    0.0  
1  0.470  0.293  0.228  0.191  0.193  0.326  0.274    0.0    0.0  

[2 rows x 35 columns]
   day    1    2    3    4    5    6    7    8    9  ...   11   12   13   14  \
0    1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
1    2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  

...

C:\Users\glogow0000\HGS-DSSAT\codebase\pyHGSDSSAT\pyHGSDSSAT\ModelInfoExchange.py:171: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(nfmb_path, skiprows = start_line, names = vars, delim_whitespace=True)


Starting processing for model: lys_e
File not found: C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1\dssat\1\100.txt. Waiting 5 seconds...
C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1\hgs
[4, 5, 6, 7, 8, 9, 10, 11, 12, 13] [1, 2, 3, 4, 5, 6, 7, 8, 9]
C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1\dssat\1\ET.OUT
    @YEAR  DOY  DAS  SRAA  TMAXA  TMINA  REFA   EOAA  EOPA   EOSA  ...   ES2D  \
0    1975  132    1  23.6   11.0    5.0 -99.0  3.496   0.0  3.496  ...  0.563   
1    1975  133    2  21.3   11.0    5.0 -99.0  3.226   0.0  3.226  ...  0.504   
2    1975  134    3  29.3   11.0    5.0 -99.0  4.223   0.0  4.223  ...  0.642   
3    1975  135    4  24.7   11.0    5.0 -99.0  3.668   0.0  3.668  ...  0.555   
4    1975  136    5  17.7   11.0    5.0 -99.0  2.817   0.0  2.817  ...  0.427   
..    ...  ...  ...   ...    ...    ...   ...    ...   ...    ...  ...    ...   
95   1975  227   96  24.0   24.0   10.0 -99.0  4.842   0.0  4.842  ...  0.047   
96   1975  228   97  22.7   20.0    2.

C:\Users\glogow0000\HGS-DSSAT\codebase\pyHGSDSSAT\pyHGSDSSAT\ModelInfoExchange.py:171: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(nfmb_path, skiprows = start_line, names = vars, delim_whitespace=True)


Starting processing for model: lys_e
File not found: C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1\dssat\1\101.txt. Waiting 5 seconds...
C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1\hgs
[4, 5, 6, 7, 8, 9, 10, 11, 12, 13] [1, 2, 3, 4, 5, 6, 7, 8, 9]
C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1\dssat\1\ET.OUT
     @YEAR  DOY  DAS  SRAA  TMAXA  TMINA  REFA   EOAA  EOPA   EOSA  ...  \
0     1975  132    1  23.6   11.0    5.0 -99.0  3.496   0.0  3.496  ...   
1     1975  133    2  21.3   11.0    5.0 -99.0  3.226   0.0  3.226  ...   
2     1975  134    3  29.3   11.0    5.0 -99.0  4.223   0.0  4.223  ...   
3     1975  135    4  24.7   11.0    5.0 -99.0  3.668   0.0  3.668  ...   
4     1975  136    5  17.7   11.0    5.0 -99.0  2.817   0.0  2.817  ...   
..     ...  ...  ...   ...    ...    ...   ...    ...   ...    ...  ...   
96    1975  228   97  22.7   20.0    2.0 -99.0  4.131   0.0  4.131  ...   
97    1975  229   98  11.4   17.0    4.0 -99.0  2.467   0.0  2.467  ...   
98    1975 

C:\Users\glogow0000\HGS-DSSAT\codebase\pyHGSDSSAT\pyHGSDSSAT\ModelInfoExchange.py:171: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(nfmb_path, skiprows = start_line, names = vars, delim_whitespace=True)


In [ ]:

# create bfx file for hgs
output_file = working_HGS_dir +'/batch.pfx'
with open(output_file, 'w') as f:
    f.write(sorted_model_names[0] + '\n')
#run grok and phgs subprocess
subprocess.run(['cmd', '/c', 'grok'], cwd=working_HGS_dir) #, stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)
subprocess.run(['cmd', '/c', 'phgs'], cwd=working_HGS_dir) #stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)
#check what is running and remove from the list
print(f"Processing model: {sorted_model_names[0]}")
# run create DRN function

print(mapping)
ie.GenerateDSSATDailyDRNHGSNodalFlux(mapping,0,model_name,working_HGS_dir,working_DSSAT_dir)


In [ ]:
# run all existing DSSAT models for HGS zones 
#for DSSAT_model_name in os.listdir(working_DSSAT_dir):
#    DSSAT_dict=working_DSSAT_dir + "\\" + DSSAT_model_name



DSSAT_dict=working_DSSAT_dir + "\\1"
print(DSSAT_dict)
exe_path = r"C:\Users\glogow0000\HGS-DSSAT\codebase\DSSAT\Coup\DSSAT\build\bin\Debug\dscsm048.exe"
process = subprocess.Popen(
    [exe_path, "C", "SWSW7501.WHX", "1"],
    cwd=DSSAT_dict
    )

print("Process started, PID:", process.pid)
#open loop for dssat subroutine

for i in range(1,final_simulation_day):
    # create bfx file for hgs
    output_file = working_HGS_dir +'/batch.pfx'
    with open(output_file, 'w') as f:
        f.write(model_name + '_day' + str(i))
    print(f"Starting processing for model: {model_name}")
    DSSAT_check = f"{i}.txt"
    full_path = os.path.join(DSSAT_dict, DSSAT_check)
    while not os.path.isfile(full_path):
        print(f"File not found: {full_path}. Waiting 5 seconds...")
        time.sleep(1)
        # If I exceeds final simulation day, break the while loop
        if i > final_simulation_day:
            print(f"Reached final simulation day for {model_name}. Moving to next model.")
            break       
    print(working_HGS_dir)
    time.sleep(1)
    # Call your processing function
    ie.GenerateNodalFluxTimeValueTableDSSATET(mapping, i, working_HGS_dir, working_DSSAT_dir)
    
    #run grok and phgs subprocess
    subprocess.run(['cmd', '/c', 'grok'], cwd=working_HGS_dir) #, stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)
    subprocess.run(['cmd', '/c', 'phgs'], cwd=working_HGS_dir) #stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)
    print(f"Processing model: {model_name} {i}")
    time.sleep(1)
    #run create DRN function
    ie.GenerateDSSATDailyDRNHGSNodalFlux(mapping,i,model_name,working_HGS_dir,working_DSSAT_dir)



In [ ]:
# libs
import subprocess
import os
import re
import pyHGSDSSAT.ModelPrep as mp
import pyHGSDSSAT.CreateMapping as cm
import pyHGSDSSAT.ModelInfoExchange as ie
import time
#variables
working_HGS_dir = r"C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1\hgs"
working_DSSAT_dir = r"C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1\dssat"
mapping_dir= r"C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1\mapping"
copuled_dir= r"C:\Users\glogow0000\HGS-DSSAT\model\coupled_v1"
# Get a fixed list of relevant model names at the start
model_name = 'lys_e'
model_files = [
    re.match(r'(lys_e_day)(\d+)\.grok', f).groups()
    for f in os.listdir(working_HGS_dir)
    if re.match(r'lys_e_day\d+\.grok', f)
]
#sort the list of model files
sorted_model_names = [f"{prefix}{int(day)}" for prefix, day in sorted(model_files, key=lambda x: int(x[1]))]
final_simulation_day= len(sorted_model_names)

